In [70]:
import pandas as pd
import numpy as np
import sklearn 
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.pipeline import Pipeline
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import pickle

In [30]:
recently_played=pd.read_csv('recently_played.csv')
event_data=pd.read_csv('event_data.csv')


user data

In [137]:
q= recently_played.iloc[20:30]
q

,artist_id,artist,genre,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10
20,0f4ne0cd4tbiZCLzpb1QoX,Developer,'minimal dub',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,0f4ne0cd4tbiZCLzpb1QoX,Developer,'minimal dub',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,7hwCBLTDeDsdc0tSeZzCFg,Coeter One,'modular techno',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,6xS3zemJD9h94iueQvGqVk,Sigha,'chillwave','electronica','future garage','indietronica','microhouse','uk bass','wonky',NaN,NaN,NaN
24,6xS3zemJD9h94iueQvGqVk,Sigha,'chillwave','electronica','future garage','indietronica','microhouse','uk bass','wonky',NaN,NaN,NaN
25,4lu50np3LdTkRL09T7x8UP,Agoria,'electronica','microhouse','minimal techno','new french touch','tech house',NaN,NaN,NaN,NaN,NaN
26,5q1QQYfkViLOUO9NzjUZCP,Artefakt,'minimal dub',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,0f4ne0cd4tbiZCLzpb1QoX,Developer,'minimal dub',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,21hEV5aKCoEtgxoDd051is,Cliche Morph,'belarusian electronic','industrial techno',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,21hEV5aKCoEtgxoDd051is,Cliche Morph,'belarusian electronic','industrial techno',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
q.mode()

,artist_id,artist,genre,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10
0,4WN5naL3ofxrVBgFpguzKo,Rudimental,'dance pop','edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop','uk bass'


In [138]:
q['all'] = q[q.columns[2:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

/tmp/ipykernel_9256/3528117245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q['all'] = q[q.columns[2:]].apply(


In [139]:
all=q['all']

In [140]:
all2= all.str.cat(sep=", ")
all2=all2.split(",")
all3 = [x[2:-1] for x in all2]
all3

['inimal dub',
 'minimal dub',
 'modular techno',
 'chillwave',
 'electronica',
 'future garage',
 'indietronica',
 'microhouse',
 'uk bass',
 'wonky',
 'chillwave',
 'electronica',
 'future garage',
 'indietronica',
 'microhouse',
 'uk bass',
 'wonky',
 'electronica',
 'microhouse',
 'minimal techno',
 'new french touch',
 'tech house',
 'minimal dub',
 'minimal dub',
 'belarusian electronic',
 'industrial techno',
 'belarusian electronic',
 'industrial techno']

source data

In [5]:
event_data.head(20)

,clubs,djs,genres
0,40seconds Club,lilith-ge,dub techno
1,40seconds Club,upsammy,dance pop
2,40seconds Club,richii,dub techno
3,40seconds Club,samkoh,chanson
4,40seconds Club,smuseb,compositional ambient
5,Abspannwerk Buchhändlerhof,nicholasmaddix,east coast hip hop
6,Abspannwerk Buchhändlerhof,djminx,electro-pop francais
7,Abspannwerk Buchhändlerhof,rom3o3-de,detroit house
8,Abspannwerk Buchhändlerhof,tereza,ethnotronica
9,Abspannwerk Buchhändlerhof,viktorkampf,electro-pop francais


group artists at each club and add all genres to each club, save new DF as DF2

In [15]:
df2= event_data.groupby('clubs')['genres'].apply(','.join).reset_index()
df2

,clubs,genres
0,40seconds Club,"dub techno,dance pop,dub techno,chanson,compos..."
1,Abspannwerk Buchhändlerhof,"east coast hip hop,electro-pop francais,detroi..."
2,Adagio Berlin,"australian dance,armenian pop,turkish edm,nouv..."
3,Arena Berlin,"happy hardcore,eurovision,melodic techno,anado..."
4,Astra Kulturhaus,"deep tropical house,arab electronic,mandible,g..."
...,...,...
56,Watergate (Berl3 F),"turkish edm,acid techno,intelligent dance musi..."
57,Weekend Club,"electro-pop francais,bergen indie,lo-fi chill,..."
58,Wilde Renate,"ninja,pop dance,gabba,experimental techno,pres..."
59,YAAM (club),"modern reggae,classical,german dance,classic f..."


In [28]:
df2.to_csv('df2.csv')

In [6]:
df2=pd.read_csv('df2.csv')

In [7]:
df2

,Unnamed: 0,clubs,genre
0,0,40seconds Club,"dub techno,dance pop,dub techno,chanson,compos..."
1,1,Abspannwerk Buchhändlerhof,"east coast hip hop,electro-pop francais,detroi..."
2,2,Adagio Berlin,"australian dance,armenian pop,turkish edm,nouv..."
3,3,Arena Berlin,"happy hardcore,eurovision,melodic techno,anado..."
4,4,Astra Kulturhaus,"deep tropical house,arab electronic,mandible,g..."
...,...,...,...
56,56,Watergate (Berl3 F),"turkish edm,acid techno,intelligent dance musi..."
57,57,Weekend Club,"electro-pop francais,bergen indie,lo-fi chill,..."
58,58,Wilde Renate,"ninja,pop dance,gabba,experimental techno,pres..."
59,59,YAAM (club),"modern reggae,classical,german dance,classic f..."


In [50]:
#df3=pd.read_csv('d3.csv')

One hot encode DF2, this also creates all the genres into sperate columns, new OHE df = out

In [8]:
out = df2.genre.str.get_dummies(',')

In [79]:
out

,acid techno,afro house,alternative dance,alternative hip hop,anadolu rock,arab electronic,arabesk,armenian hip hop,armenian pop,aussietronica,...,turkish folk,turkish hip hop,turkish pop,turkish rock,turkish trap,uk dancehall,urbano espanol,vapor twitch,violin,york indie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


join df2 and out to get all clubs and columns

In [9]:
df4= df2.join(out)

extract column names to a list

In [121]:
genrelist = df4.columns

take only columns needed... may not need to join df2 and out, and can skip this step

In [125]:
genrelist2 = genrelist[3:]

In [126]:
genrelist2

Index(['acid techno', 'afro house', 'alternative dance', 'alternative hip hop',
       'anadolu rock', 'arab electronic', 'arabesk', 'armenian hip hop',
       'armenian pop', 'aussietronica',
       ...
       'turkish folk', 'turkish hip hop', 'turkish pop', 'turkish rock',
       'turkish trap', 'uk dancehall', 'urbano espanol', 'vapor twitch',
       'violin', 'york indie'],
      dtype='object', length=140)

create new DF with all genre names as the columns.. needed to fit user data to model

In [127]:
pred_df = pd.DataFrame(columns=genrelist2)

In [128]:
pred_df

,acid techno,afro house,alternative dance,alternative hip hop,anadolu rock,arab electronic,arabesk,armenian hip hop,armenian pop,aussietronica,...,turkish folk,turkish hip hop,turkish pop,turkish rock,turkish trap,uk dancehall,urbano espanol,vapor twitch,violin,york indie


DF4 has clubs, a coloumn with all club genres + 140 columns of individual genres. need to get full genre list from spotify and use it here instead

Create x and y for train test split and use in RandomForestClassifier model

In [15]:
X= df4.drop(['Unnamed: 0', 'genre','clubs'], axis=1)
y= df4['clubs']

In [16]:
X

,acid techno,afro house,alternative dance,alternative hip hop,anadolu rock,arab electronic,arabesk,armenian hip hop,armenian pop,aussietronica,...,turkish folk,turkish hip hop,turkish pop,turkish rock,turkish trap,uk dancehall,urbano espanol,vapor twitch,violin,york indie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
y

0                  40seconds Club
1      Abspannwerk Buchhändlerhof
2                   Adagio Berlin
3                    Arena Berlin
4                Astra Kulturhaus
                 ...             
56            Watergate (Berl3 F)
57                   Weekend Club
58                   Wilde Renate
59                    YAAM (club)
60                Zur Mobelfabrik
Name: clubs, Length: 61, dtype: object

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42 )

In [25]:
X_test

,acid techno,afro house,alternative dance,alternative hip hop,anadolu rock,arab electronic,arabesk,armenian hip hop,armenian pop,aussietronica,...,turkish folk,turkish hip hop,turkish pop,turkish rock,turkish trap,uk dancehall,urbano espanol,vapor twitch,violin,york indie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
46,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
31,0,0,0,0,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
34,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
49,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


Created model, need to look at hyper parameters and decided which is best to choose

In [22]:

clf=RandomForestClassifier(max_depth=5 , random_state=42, bootstrap=True  )

In [23]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, random_state=42)

selecting 1 row from test data to predict and reshape to fit model

In [28]:
test=X_test.iloc[0]
test=np.array(test).reshape(1,-1)

In [29]:
clf.predict(test)

/home/nathan/anaconda3/envs/party/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([' Sisyphos (club)'], dtype=object)

In [71]:
encoder=OneHotEncoder()

In [113]:
test= encoder.fit_transform(df2[['genre']])

In [115]:
test

<61x61 sparse matrix of type '<class 'numpy.float64'>'
	with 61 stored elements in Compressed Sparse Row format>

create new df all3 from all3 list created in the userdata section, names the columns as the genres 

In [141]:
all3=pd.DataFrame(all3)
all3.columns=["genre"]
all3 = all3['genre'].str.cat(sep=', ')

In [118]:
all3

'dm, pop, pop dance, tropical house, modern alternative rock, modern rock, pop, pop rap, pop rock, rock, stomp pop, ohio hip hop, modern alternative rock, modern rock, pop, pop rap, pop rock, rock, stomp pop, modern alternative rock, modern rock, pop, pop rap, pop rock, rock, stomp pop, dance pop, edm, house, pop, pop dance, tropical house, uk dance, uk funky, uk pop, dance pop, pop, pop dance, post-teen pop, tropical house, uk contemporary r&b, uk pop, dance pop, edm, house, pop, pop dance, tropical house, uk dance, uk funky, uk pop, house, tropical house, dance pop, edm, house, pop, pop dance, tropical house, uk dance, uk funky, uk pop, dance pop, edm, house, pop, pop dance, tropical house, uk dance, uk funky, uk pop, dance pop, edm, house, pop, pop dance, tropical house, uk dance, uk funky, uk pop, organic electronic, deep euro house, hamburg electronic, minimal techno, tech house, funky tech house, latin tech house, deep techno, ambient techno, dub techno, float house, intelligent 

create new list of genres called result from pref_df and all3 to make sure the same unique genres are included 

In [142]:
result = []

for columname in pred_df.columns:
    if columname in all3:
        result.append(1)
    else:
        result.append(0)

In [143]:
pred_df

,acid techno,afro house,alternative dance,alternative hip hop,anadolu rock,arab electronic,arabesk,armenian hip hop,armenian pop,aussietronica,...,turkish folk,turkish hip hop,turkish pop,turkish rock,turkish trap,uk dancehall,urbano espanol,vapor twitch,violin,york indie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
pred_df.loc[len(pred_df.index)] = result

q is selection on songs from recently played

In [148]:
q

,artist_id,artist,genre,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10,all
20,0f4ne0cd4tbiZCLzpb1QoX,Developer,'minimal dub',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'minimal dub'
21,0f4ne0cd4tbiZCLzpb1QoX,Developer,'minimal dub',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'minimal dub'
22,7hwCBLTDeDsdc0tSeZzCFg,Coeter One,'modular techno',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'modular techno'
23,6xS3zemJD9h94iueQvGqVk,Sigha,'chillwave','electronica','future garage','indietronica','microhouse','uk bass','wonky',NaN,NaN,NaN,"'chillwave', 'electronica', 'future garage', '..."
24,6xS3zemJD9h94iueQvGqVk,Sigha,'chillwave','electronica','future garage','indietronica','microhouse','uk bass','wonky',NaN,NaN,NaN,"'chillwave', 'electronica', 'future garage', '..."
25,4lu50np3LdTkRL09T7x8UP,Agoria,'electronica','microhouse','minimal techno','new french touch','tech house',NaN,NaN,NaN,NaN,NaN,"'electronica', 'microhouse', 'minimal techno',..."
26,5q1QQYfkViLOUO9NzjUZCP,Artefakt,'minimal dub',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'minimal dub'
27,0f4ne0cd4tbiZCLzpb1QoX,Developer,'minimal dub',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'minimal dub'
28,21hEV5aKCoEtgxoDd051is,Cliche Morph,'belarusian electronic','industrial techno',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"'belarusian electronic', 'industrial techno'"
29,21hEV5aKCoEtgxoDd051is,Cliche Morph,'belarusian electronic','industrial techno',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"'belarusian electronic', 'industrial techno'"


In [145]:
pred_df

,acid techno,afro house,alternative dance,alternative hip hop,anadolu rock,arab electronic,arabesk,armenian hip hop,armenian pop,aussietronica,...,turkish folk,turkish hip hop,turkish pop,turkish rock,turkish trap,uk dancehall,urbano espanol,vapor twitch,violin,york indie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


raw 0 is the 20 most recently_played songs
row 1 is 20->30 most recently played song

In [147]:
clf.predict(pred_df.loc[[1]])

array([' M-BIA'], dtype=object)

In [102]:
all5.loc['genre'] = all3

In [119]:
all5

,genre
genre,"dm, pop, pop dance, tropical house, modern alt..."


In [117]:
all4=encoder.transform(all5['genre'])
all4

/home/nathan/anaconda3/envs/party/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


ValueError: X does not contain any features, but OneHotEncoder is expecting 1 features

In [ ]:
query = recently_played.iloc[:30]
query =query.dropna(thresh=5, axis=1)
query

First I need to Feature engineer the data with label encoding and onehotencoding

random forest classifier